In [1]:
import os 
import sys
import datetime
import csv 

import cirq
import openfermion
import numpy as np
Pi=3.1415


In [2]:
from functools import partial

## anzats

In [19]:
class Anzats():
    def __init__(self, length, gamma, beta):

        # initialize circuit
        circuit = cirq.Circuit()
        qubits = cirq.LineQubit.range(length)

        for i in range(length):
            circuit.append(cirq.H(qubits[i]))

        # add gamma circuit
        for index in range(gamma.size):
            # add gates on 2n and 2n+1
            for i in range(1, length, 2):
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
                    )
                # circuit.append(
                #     cirq.ZZPowGate(
                #         exponent=gamma[index]/Pi, global_shift=0.0).on(
                #             qubits[i], qubits[(i+1)%length]
                #     )
                # )
            # add gates on 2n-1 and 2n
            for i in range(0, length, 2):
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
                    )
                # circuit.append(
                #     cirq.ZZPowGate(
                #         exponent=gamma[index]/Pi, global_shift=0.0).on(
                #             qubits[i], qubits[(i+1)%length]
                #         )
                # )

            # add beta circuit
            for i in range(length):
                circuit.append(
                    cirq.XPowGate(
                        exponent=beta[index]*2/Pi, global_shift=0.0).on(
                        qubits[i])
                )

        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta




class AnzatsForToricCode():
    def __init__(self, length, gamma, beta):

        # initialize circuit
        circuit = cirq.Circuit()
        qubits = cirq.GridQubit.range(length, length)

        for i in range(length):
            for j in range(length):
                circuit.append(cirq.H([qubits[i], qubits[j]]))

        # add gamma circuit
        for index in range(gamma.size):
            # add gates on 2n and 2n+1
            # for i in range(1, length, 2):
            #     circuit.append(
            #         cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
            #         )
            #     # circuit.append(
            #     #     cirq.ZZPowGate(
            #     #         exponent=gamma[index]/Pi, global_shift=0.0).on(
            #     #             qubits[i], qubits[(i+1)%length]
            #     #     )
            #     # )
            # # add gates on 2n-1 and 2n
            # for i in range(0, length, 2):
            #     circuit.append(
            #         cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
            #         )
                # circuit.append(
                #     cirq.ZZPowGate(
                #         exponent=gamma[index]/Pi, global_shift=0.0).on(
                #             qubits[i], qubits[(i+1)%length]
                #         )
                # )

            # add beta circuit
            for i in range(length):
                for j in range(length):
                    circuit.append(
                        cirq.XPowGate(
                            exponent=beta[index]*2/Pi, global_shift=0.0).on(
                            qubits[i], qubits[j])
                    )

        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta




## expectation

In [4]:
class ToricArgs():
    def __init__(self, length, g):
        self.length = length
        self.g = g


def get_expectation_critical_state(function_args, gamma, beta):
    anzats = AnzatsForToricCode(function_args.length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    vector = cirq.final_state_vector(circuit)

    value = 0 + 0j
    for i in range(function_args.length):
        circuit = anzats.circuit.copy()
        circuit.append(cirq.Z(qubits[i]))
        circuit.append(cirq.Z(qubits[(i+1)%function_args.length]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    for i in range(function_args.length):
        circuit = anzats.circuit.copy()
        # circuit.append(function_args.g * cirq.X(qubits[i]))
        circuit.append(cirq.XPowGate(exponent=function_args.g ).on(qubits[i]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    return value

class TFIMStateArgs():
    def __init__(self, length, g):
        self.length = length
        self.g = g


def get_expectation_critical_state(function_args, gamma, beta):
    anzats = Anzats(function_args.length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    vector = cirq.final_state_vector(circuit)

    value = 0 + 0j
    for i in range(function_args.length):
        circuit = anzats.circuit.copy()
        circuit.append(cirq.Z(qubits[i]))
        circuit.append(cirq.Z(qubits[(i+1)%function_args.length]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    for i in range(function_args.length):
        circuit = anzats.circuit.copy()
        # circuit.append(function_args.g * cirq.X(qubits[i]))
        circuit.append(cirq.XPowGate(exponent=function_args.g ).on(qubits[i]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    return value

def get_expectation_ghz(length, gamma, beta):
    anzats = Anzats(length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    vector = cirq.final_state_vector(circuit)

    value = 0 + 0j
    for i in range(length):
        circuit = anzats.circuit.copy()
        circuit.append(cirq.Z(qubits[i]))
        circuit.append(cirq.Z(qubits[(i+1)%length]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    return value


def get_expectation_ghz_l4(gamma: np.array, beta: np.array):
    return get_expectation_ghz(4, gamma, beta)

def get_expectation_ghz_l8(gamma: np.array, beta: np.array):
    return get_expectation_ghz(8, gamma, beta)

def get_expectation_ghz_l10(gamma: np.array, beta: np.array):
    return get_expectation_ghz(10, gamma, beta)

def get_expectation_ghz_l12(gamma: np.array, beta: np.array):
    return get_expectation_ghz(12, gamma, beta)

def get_expectation_ghz_l14(gamma: np.array, beta: np.array):
    return get_expectation_ghz(14, gamma, beta)

def get_expectation_ghz_l16(gamma: np.array, beta: np.array):
    return get_expectation_ghz(16, gamma, beta)


## optimization

In [5]:

def get_gradient(function, gamma: np.array, beta: np.array, delta_gamma, delta_beta, iter):
    grad_gamma = np.zeros_like(gamma)
    grad_beta  = np.zeros_like(beta)
    gamma_edge = gamma
    beta_edge  = beta
    # initial gamma, beta?
    
    if not (gamma.size == beta.size):
        return 1

    for index in range(gamma.size):
        center = gamma[index]
        gamma_edge[index] = gamma[index] - delta_gamma
        e1 = function(gamma=gamma_edge, beta=beta)
        gamma_edge[index] = gamma[index] + delta_gamma
        e2 = function(gamma=gamma_edge, beta=beta)
        grad_gamma[index] = (e2.real-e1.real)/(2*delta_gamma)
        gamma[index] = center

        center = beta[index]
        beta_edge[index] = beta[index] - delta_beta
        e1 = function(gamma=gamma, beta=beta_edge)
        beta_edge[index] = beta[index] + delta_beta
        e2 = function(gamma=gamma, beta=beta_edge)
        grad_beta[index] = (e2.real-e1.real)/(2*delta_beta)
        beta[index] = center
    
    return grad_gamma, grad_beta


def optimize_by_gradient_descent(function, initial_gamma: np.array, initial_beta: np.array, alpha, delta_gamma, delta_beta, iteration, Figure=True,filepath=""):
    gamma, beta = initial_gamma, initial_beta

    textlines = []
    headline = ["iter", "energy"]
    for p in range(int(len(initial_gamma))):
        headline.append("gamma[{}]".format(p))
        headline.append("bata[{}]".format(p))
    print(headline)
    textlines.append(headline)

    for iter in range(int(iteration)):
        # it is complex for me to set get_gradient for two optical parameter_vector
        grad_gamma, grad_beta = get_gradient(function, gamma, beta, delta_gamma, delta_beta, iter)
        gamma -= alpha * grad_gamma
        beta  -= alpha * grad_beta
        energy = function(gamma=gamma, beta=beta)

        record = [iter, energy]
        for index in range(gamma.size):
            record.append(gamma[index])
            record.append(beta[index])
        textlines.append(record)
        print(record)
    
    if len(filepath)>0:
        with open(filepath, mode='a') as f:
            writer = csv.writer(f)
            for i, textline in enumerate(textlines):
                writer.writerow(textline)
                # f.write("{}\n".format(textline))

    return gamma, beta


## case: length 4, static gamma and beta

In [6]:
length = 4
g = 0.8
gamma = np.array([0.5, 0.4, 0.3, 0.2])
beta  = np.array([0.7, 0.5, 0.3, 0.1])
function_args = TFIMStateArgs(length, g)

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
# print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_critical_state(function_args,gamma, beta)
print("expectation on ghz: {}".format(value))

measurements: (no measurements)

qubits: (cirq.LineQubit(3), cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2))
output vector: [ 0.21862863-0.3136721j   0.01913893-0.24795502j  0.01913894-0.24795511j
  0.00876991-0.07031819j  0.01913892-0.24795505j -0.1374358 -0.278366j
  0.00876982-0.0703182j   0.01913891-0.24795498j  0.01913891-0.24795498j
  0.00876982-0.0703182j  -0.1374358 -0.278366j    0.01913892-0.24795505j
  0.00876991-0.07031819j  0.01913894-0.24795511j  0.01913893-0.24795502j
  0.21862863-0.3136721j ]

phase:
output vector: |⟩
norm: (1+0j)
expectation on ghz: (-3.4369701966643333+0.31241975724697113j)


## case: length 8, static gamma and beta (paper value)

In [7]:
length = 8
gamma = np.array([0.2496, 0.4808, 0.5260, 0.4503])
beta  = np.array([0.6845, 0.6559, 0.6048, 0.3180])
g = 0.5
# g = 0.4
function_args = TFIMStateArgs(length, g)

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
# print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_critical_state(function_args, gamma, beta)
print("expectation on ghz: {}".format(value))

measurements: (no measurements)

qubits: (cirq.LineQubit(7), cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4), cirq.LineQubit(5), cirq.LineQubit(6))
output vector: [0.4591058 -4.49337065e-03j 0.12919495-1.25825033e-03j
 0.12919495-1.25830155e-03j 0.07737847-7.35415146e-04j
 0.12919495-1.25823729e-03j 0.04038734-3.95270996e-04j
 0.07737848-7.35405833e-04j 0.06066925-5.82892448e-04j
 0.12919493-1.25823729e-03j 0.03783053-3.70066147e-04j
 0.04038735-3.95267271e-04j 0.02739703-2.70092394e-04j
 0.0773785 -7.35389534e-04j 0.02739705-2.70094723e-04j
 0.06066927-5.82899433e-04j 0.05631589-5.69676980e-04j
 0.12919495-1.25824474e-03j 0.03746548-3.55740078e-04j
 0.03783052-3.70071735e-04j 0.0245581 -2.27298122e-04j
 0.04038734-3.95286363e-04j 0.01372327-1.31525332e-04j
 0.02739706-2.70090764e-04j 0.02465377-2.33101659e-04j
 0.0773785 -7.35399313e-04j 0.02455809-2.27314420e-04j
 0.02739705-2.70107295e-04j 0.02209693-2.07863282e-04j
 0.06066924-5.8288779

expectation on ghz: (-11.689059972763062+1.4373957514762878j)


## case: length 4, optimize beta and gamma

In [8]:
length = 4
initial_gamma = np.array([0.1, 0.2])
initial_beta  = np.array([0.1, 0.2])
g = 1.0
function_args = TFIMStateArgs(length, g)

iteration = 10
alpha = 0.1
delta_gamma = 0.001
delta_beta  = 0.001

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_critical_state, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]']
[0, (-3.333689719438553+0j), -0.04722943305969238, 0.2243487000465393, -0.04554282426834105, 0.42125691175460817]
[1, (-3.1210728138685226+0j), 0.27399862408638, 0.2961998641490936, 0.458268415927887, 0.4917863428592682]
[2, (-4.527633816003799+0j), 0.2285433769226074, 0.7764620542526245, 0.25658939480781556, 0.38910765647888185]
[3, (-2.30567230284214+0j), 0.0017216265201568381, 0.6361683666706085, 0.5638945519924163, 0.38574148416519166]
[4, (0.5406368523836136+0j), 0.6504168570041656, 0.6364619195461273, 0.0021438717842101607, 0.39478201866149903]
[5, (0.7603021264076233+0j), 0.7273269653320312, 0.5223331809043884, 0.7178742110729217, 0.2823177397251129]
[6, (-3.875464081764221+0j), 0.1270724296569824, -0.05242844820022585, 0.27304921746253963, 0.4054952085018158]
[7, (0.4968729615211487+0j), -0.31561275720596316, -0.004598701000213645, -0.15385414958000188, 0.4107508480548859]
[8, (-4.306087590754032+0j), -0.058174335

## case: length 8, optimize beta and gamma

In [9]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
g = 1.0
function_args = TFIMStateArgs(length, g)

iteration = 10
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

# print("exact energy: ")
# print(-1.28*length)

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_critical_state, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]', 'gamma[2]', 'bata[2]', 'gamma[3]', 'bata[3]']
[0, (-8.64386373758316+1.4901161193847656e-08j), 0.4595354497432709, 0.528499960899353, 0.519349604845047, 0.4975678324699402, 0.504250556230545, 0.5230531394481659, 0.4914120137691498, 0.5004075467586517]
[1, (-9.043448567390442-7.450580596923828e-09j), 0.42679089307785034, 0.5465282797813416, 0.5324256718158722, 0.49900922179222107, 0.5084276497364044, 0.5437791645526886, 0.4794262647628784, 0.49144837260246277]
[2, (-9.293071389198303-1.4901161193847656e-08j), 0.4017718434333801, 0.5583362579345703, 0.5400950014591217, 0.5057299435138702, 0.5132593512535095, 0.5596082210540771, 0.46912866830825806, 0.47946083545684814]
[3, (-9.461586326360703-1.4901161193847656e-08j), 0.3832304775714874, 0.5671529769897461, 0.5438277423381805, 0.5168471038341522, 0.5187832117080688, 0.5709965825080872, 0.46223896741867065, 0.46745720505714417]
[4, (-9.592377454042435+2.9802322387695312e-08j

## case: length 8, optimize beta and gamma b

In [10]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
g = 1.0
function_args = TFIMStateArgs(length, g)

iteration = 20
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

# print("energy in paper eyeseeing value: ")
# print(-1.28*length)

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_critical_state, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]', 'gamma[2]', 'bata[2]', 'gamma[3]', 'bata[3]']
[0, (-8.64386373758316+1.4901161193847656e-08j), 0.4595354497432709, 0.528499960899353, 0.519349604845047, 0.4975678324699402, 0.504250556230545, 0.5230531394481659, 0.4914120137691498, 0.5004075467586517]
[1, (-9.043448567390442-7.450580596923828e-09j), 0.42679089307785034, 0.5465282797813416, 0.5324256718158722, 0.49900922179222107, 0.5084276497364044, 0.5437791645526886, 0.4794262647628784, 0.49144837260246277]
[2, (-9.293071389198303-1.4901161193847656e-08j), 0.4017718434333801, 0.5583362579345703, 0.5400950014591217, 0.5057299435138702, 0.5132593512535095, 0.5596082210540771, 0.46912866830825806, 0.47946083545684814]
[3, (-9.461586326360703-1.4901161193847656e-08j), 0.3832304775714874, 0.5671529769897461, 0.5438277423381805, 0.5168471038341522, 0.5187832117080688, 0.5709965825080872, 0.46223896741867065, 0.46745720505714417]
[4, (-9.592377454042435+2.9802322387695312e-08j

## case: length 16, static beta and gamma (paper value)

In [11]:
length = 16
p = 8
initial_gamma = np.array([0.3790,0.5638,0.9046,0.6738,0.8616,0.5624,0.5224,0.4119])
initial_beta  = np.array([0.5622,0.7101,0.3210,0.8377,0.4004,0.9450,0.6466,0.5172])
g = 1.0
function_args = TFIMStateArgs(length, g)


# print("energy in paper eyeseeing value: ")
# print(-1.275*length)
print(get_expectation_critical_state(function_args, initial_gamma, initial_beta))

print(gamma, beta)

(-20.404560446739197-1.4901161193847656e-07j)
[0.28962615 0.52295911 0.55395338 0.46857613] [0.66170695 0.64693186 0.61849791 0.36199349]


## playground

In [12]:
L = 10
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(L)

for i in range(1, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
        )
    )


for i in range(2, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
            )
    )


for i in range(L):
    circuit.append(
        cirq.XPowGate(
            exponent=0.5, global_shift=0.0).on(
            qubits[i])
    )


# for i in range(1, L, 2):
#     circuit.append(cirq.Z(qubits[i-1]))
#     circuit.append(cirq.Z(qubits[i]))


# for i in range(2, L, 2):
#     circuit.append(cirq.Y(qubits[i-1]))
#     circuit.append(cirq.Y(qubits[i]))


# for i in range(L):
#     circuit.append(cirq.X(qubits[i]))



print(circuit)


0: ───ZZ───────X^0.5────────────
      │
1: ───ZZ^0.5───ZZ───────X^0.5───
               │
2: ───ZZ───────ZZ^0.5───X^0.5───
      │
3: ───ZZ^0.5───ZZ───────X^0.5───
               │
4: ───ZZ───────ZZ^0.5───X^0.5───
      │
5: ───ZZ^0.5───ZZ───────X^0.5───
               │
6: ───ZZ───────ZZ^0.5───X^0.5───
      │
7: ───ZZ^0.5───ZZ───────X^0.5───
               │
8: ───ZZ───────ZZ^0.5───X^0.5───
      │
9: ───ZZ^0.5───X^0.5────────────


In [13]:
value = 0 + 0j

# circuit2 = circuit
# anzats2 = Anzats(length, gamma, beta)
# circuit2 = anzats2.circuit
# qubits = anzats2.qubits
# circuit2.append(
#         cirq.ZZPowGate(
#             exponent=1, global_shift=0.0).on(
#                 qubits[1], qubits[2]
#         ))
# print(circuit2)
# result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
# vector2 = cirq.final_state_vector(circuit2)
# value -= np.dot(vector2.conjugate(), vector)

# print(value)

# # for i in range(length-1):
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[i], qubits[i+1]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)
# else:
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[length-1], qubits[0]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)

In [14]:
a = np.array([1,3])
a[1]

3

In [15]:
print(vector)

[0.4591058 -4.49337065e-03j 0.12919496-1.25825405e-03j
 0.12919495-1.25825033e-03j 0.07737847-7.35387206e-04j
 0.12919495-1.25830155e-03j 0.04038734-3.95298004e-04j
 0.07737847-7.35415146e-04j 0.06066927-5.82884997e-04j
 0.12919495-1.25823729e-03j 0.03783051-3.70055437e-04j
 0.04038734-3.95270996e-04j 0.02739706-2.70117074e-04j
 0.07737848-7.35405833e-04j 0.02739704-2.70104036e-04j
 0.06066925-5.82892448e-04j 0.05631588-5.69688156e-04j
 0.12919493-1.25823729e-03j 0.03746548-3.55772674e-04j
 0.03783053-3.70066147e-04j 0.0245581 -2.27299519e-04j
 0.04038735-3.95267271e-04j 0.01372327-1.31545588e-04j
 0.02739703-2.70092394e-04j 0.02465377-2.33075581e-04j
 0.0773785 -7.35389534e-04j 0.02455811-2.27298588e-04j
 0.02739705-2.70094723e-04j 0.02209694-2.07859091e-04j
 0.06066927-5.82899433e-04j 0.02465375-2.33095139e-04j
 0.05631589-5.69676980e-04j 0.06066924-5.82894310e-04j
 0.12919495-1.25824474e-03j 0.03783055-3.70055437e-04j
 0.03746548-3.55740078e-04j 0.0245581 -2.27305107e-04j
 0.0378305

In [16]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

p = 1
initial_gamma = np.array([-3.0 for i in range(p)])
initial_beta  = np.array([4.0 for i in range(p)])

alpha = 0.1
delta_gamma = 0.0001
delta_beta = 0.0001
iteration = 100

final_gamma, final_beta = optimize_by_gradient_descent(function_norm, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration, Figure=True)

print(final_gamma, final_beta)

['iter', 'energy', 'gamma[0]', 'bata[0]']
[0, 20.250009000048443, -2.6999950000002855, 3.6000049999995696]
[1, 16.402515390162296, -2.4299904999994766, 3.2400094999975835]
[2, 13.286044756250423, -2.186986449998926, 2.9160135499979276]
[3, 10.761702813850011, -1.9682828049986085, 2.6244171949971715]
[4, 8.71698518447511, -1.771449524498145, 2.3619804754972904]
[5, 7.060763314251251, -1.5942995720487048, 2.1257874279467757]
[6, 5.71922306798135, -1.4348646148437645, 1.9132136851514012]
[7, 4.632574990255437, -1.2913731533593626, 1.721897316635964]
[8, 3.7523896168740816, -1.1622308380235786, 1.5497125849721591]
[9, 3.039439077053899, -1.0460027542212842, 1.3947463264749569]
[10, 2.461948791156269, -0.9413974787991748, 1.2552766938276303]
[11, 1.9941813457998523, -0.8472527309192319, 1.1297540244450364]
[12, 1.6152894326596015, -0.7625224578273722, 1.0167836220005242]
[13, 1.3083867288548008, -0.6862652120446207, 0.9151102598005076]
[14, 1.0597953100277273, -0.617633690840111, 0.82360423

In [17]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

initial_gamma = np.array([3.0, 2.0])
initial_beta = np.array([4.0, 5.0])

final_gamma, final_beta = optimize_by_gradient_descent(
    function=function_norm,
    initial_gamma=initial_gamma,
    initial_beta=initial_beta,
    alpha=0.1,
    delta_gamma=0.001,
    delta_beta=0.001,
    iteration=100,
    Figure=True)
print(final_gamma, final_beta)
print(function_norm(gamma, beta))

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]']
[0, 43.741260009996445, 2.7000499999998624, 3.6000500000000955, 1.8000499999999846, 4.500049999999618]
[1, 35.43155463610003, 2.4300949999999517, 3.2400950000003945, 1.6200949999998642, 4.0500949999997715]
[2, 28.700579899441102, 2.1871354999997124, 2.9161355000003795, 1.4581354999997558, 3.6451354999999808]
[3, 23.248388317329418, 1.9684719499996035, 2.6245719500004725, 1.3123719499998003, 3.2806719500002757]
[4, 18.83202129493629, 1.7716747549995944, 2.3621647550002365, 1.1811847549998404, 2.9526547549999904]
[5, 15.254681350009095, 1.5945572794996394, 2.1259982795000596, 1.0631162794997522, 2.657439279499947]
[6, 12.35696160350955, 1.4351515515496267, 1.9134484515502557, 0.9568546515497083, 2.391745351550174]
[7, 10.00974165684386, 1.2916863963946952, 1.7221536063953344, 0.8612191863947665, 2.1526208163951743]
[8, 8.108433243244505, 1.162567756755248, 1.5499882457558698, 0.7751472677553366, 1.9374087347556923]
[9, 6.56

In [18]:
length = 6
gamma = np.array([0.1, 0.1])
beta = np.array([0.1, 0.1])

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
qubits = anzats.qubits
vector = cirq.final_state_vector(circuit)

ham_nd = np.zeros((length, length), dtype=np.complex128)

print(cirq.Z(qubits[-1]))
for i in range(length-1):
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[i]))
    circuit_ZiZj.append(cirq.Z(qubits[i+1]))
    ham_nd += cirq.unitary(circuit_ZiZj)
else:
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[length-1]))
    circuit_ZiZj.append(cirq.Z(qubits[0]))
    ham_nd += cirq.unitary(circuit_ZiZj)
    
print(ham_nd)


# def get_expectation_ZiZj(length, beta, gamma, qubit_ZZ_on_i, qubit_ZZ_on_j, print_circuit=False):
#     """
#     controls of append on circuit are accumulated.
#     we have to make circuits on each hamiltonian terms.
#     """
#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     vector = cirq.final_state_vector(circuit)

#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_i]))
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_j]))
#     if print_circuit:
#         print(circuit)
#     simulator = cirq.Simulator()
#     # result2 = simulator.simulate(circuit, qubit_order=qubits)
#     vector2 = cirq.final_state_vector(circuit)
#     return np.dot(vector2.conjugate(), vector)


# def get_expectation_ghz_l4(gamma: np.array, beta: np.array):
#     length = 4
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


# def get_expectation_ghz_l8(gamma: np.array, beta: np.array):
#     length = 8
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


Z(q(5))


ValueError: operands could not be broadcast together with shapes (6,6) (4,4) (6,6) 